In [1]:
import pandas as pd 
import numpy as np
import os
from datetime import datetime, timedelta
from abc import ABC, abstractmethod
from time import sleep
from IPython.display import clear_output

In [2]:
class Data:
    def __init__(self, filepath):
        self.filepath = filepath 
        self.data, self.corrupt_files = self.load_data(filepath)

    @staticmethod
    def create_filepath(folder_path): 
        corrupt_files = pd.DataFrame(columns=['Symbol', 'Filepath'])
        stocks = pd.DataFrame(columns=['Symbol', 'Filepath'])

        for filename in os.listdir(folder_path):
            filepath = os.path.join(folder_path, filename)
            try:
                data = pd.read_excel(filepath)
                file_frame = pd.DataFrame({"Symbol": [filename], "Filepath": [filepath]})
                if data.shape[1] == 6:
                    stocks = pd.concat([stocks, file_frame], ignore_index=True)
                else:
                    corrupt_files = pd.concat([corrupt_files, file_frame], ignore_index=True)

            except:
                data = pd.DataFrame({"Symbol": [filename], "Filepath": [filepath]})
                corrupt_files = pd.concat([corrupt_files, file_frame], ignore_index=True)

        return stocks, corrupt_files

    @staticmethod
    def load_data(filepath):
        corrupt_files = pd.DataFrame(columns=["Files"])
        df = pd.DataFrame(columns=["Data", "Close", "Open", "High", "Low", "Volume"])
        try:
            data = pd.read_excel(filepath)
            if data.shape[1] == 6:
                df = pd.DataFrame(data, columns=["Date", "Close", "Open", "High", "Low", "Volume"])
                df['Delta Price'] = df["Close"] - df["Open"]
                df['Velocity'] = 0
                for i in df.index:
                    if df.loc[i, 'Delta Price'] != 0:
                        df.loc[i, 'Velocity'] = (df.loc[i, 'High'] - df.loc[i, 'Low']) * df.loc[i, 'Delta Price'] / abs(df.loc[i, 'Delta Price'])
                
                df['Momentum'] = df['Volume'] * df['Velocity']

                df['Delta Momentum'] = df['Momentum'].diff().fillna(0)

                df.dropna(inplace=True)

                df.reset_index(drop=True, inplace=True)
                # print(df)
            else:
                data = pd.DataFrame({"Files": [filepath]})
                corrupt_files = pd.concat([corrupt_files, data])
        except ValueError:
            data = pd.DataFrame({"Files": [filepath]})
            corrupt_files = pd.concat([corrupt_files, data])
        return df, corrupt_files

    @staticmethod
    def excel_date_to_datetime(excel_serial_date):
        excel_start_date = datetime(1899, 12, 30)
        delta_days = timedelta(days=excel_serial_date)
        date_time = excel_start_date + delta_days 
        return date_time
    
    class Trend:
        def __init__(self, data):
            self.data = data
            self.trend, self.start_index, self.start_value, self.index_wave5, self.index_waveC, self.trend_direction, self.wave_min, self.wave_max, self.waveC, self.wave5 = self.find_trend()
            self.impulse_up = self.trend[0]
            self.start_value = self.trend[1]
            self.start_index = self.trend[2]
            self.wave_length = abs(self.wave_max - self.wave_min)
            self.proportion = 1
            self.value = self.start_value
            self.temp_value = self.value
            self.wave_type = "Initial"
            self.wave_identity = "Initial"
            self.wave_history = pd.DataFrame()
    
        def find_trend(self):
            # Drop rows with NaN values
            self.data.dropna(inplace=True)

            # Now find the index of the row with the minimum price
            wave5 = self.data["High"].max()
            high_values = self.data["High"].values
            index_wave5 = np.where(high_values == wave5)[0].tolist()

            # Ensure there's at least one index returned
            if len(index_wave5) > 0:
                index_wave5 = index_wave5[0]
                # print("Index of Largest Cycle Wave 5:", index_wave5)
            else:
                print("No index found for Wave 5.")

            # Now find the index of the row with the minimum price
            waveC = self.data["Low"].min()
            index_waveC = self.data.index[self.data["Low"] == waveC].tolist()

            # Ensure there's at least one index returned
            if len(index_waveC) > 0:
                index_waveC = index_waveC[0]
                # print("Index of Largest Cycle Wave C:", index_waveC)
            else:
                print("No index found for Wave C.")

            # Compare the indices of Wave C and Wave 5 to get a sense of the general trend
            if index_waveC == index_wave5: 
                print("Time to cross the bridge we said we would cross when we came to it.")
                print(self.data.loc[index_wave5])
                if self.data.loc[index_wave5, "Close"] > self.data.loc[index_waveC, "Open"]:
                    impulseUp = False
                    start_index = index_wave5 + 1
                    start_value = wave5
                    wave_min = waveC
                    wave_max = wave5
                    trend = [impulseUp, start_value, start_index]
                else:
                    impulseUp = True
                    start_index = index_waveC + 1
                    start_value = waveC
                    wave_min = waveC
                    wave_max = wave5
                    trend = [impulseUp, start_value, start_index]                                       
            elif index_waveC > index_wave5 or index_wave5 == len(self.data) - 1:
                impulseUp = True
                start_index = index_waveC 
                start_value = waveC
                wave_min = waveC
                wave_max = wave5
                trend = [impulseUp, start_value, start_index]
                # pd.DataFrame({'Impulse Up': [impulseUp]}, {'Start Value': [waveC]}, {'Start Index': [index_waveC]})
            elif index_wave5 > index_waveC or index_waveC == len(self.data) - 1:
                impulseUp = False
                start_index = index_wave5
                start_value = wave5
                wave_min = waveC
                wave_max = wave5
                trend = [impulseUp, start_value, start_index]
                # pd.DataFrame({'Corrective Down': [correctiveDown]}, {'Start Value': [wave5]}, {'Start Index': [index_waveC]})
            else:
                "There is an unforeseen fourth option, apparently. Please contact the developer."
            
            if impulseUp:
                trend_direction = "Down"
            else:
                trend_direction = "Up"
            
            return trend, start_index, start_value, index_wave5, index_waveC, trend_direction, wave_min, wave_max, waveC, wave5
        
        def set_initial_history(self):
            row_wave5 = pd.DataFrame({self.wave_identity: [self.wave5]}, index=[self.index_wave5])
            row_waveC = pd.DataFrame({self.wave_identity: [self.waveC]}, index=[self.index_waveC])

            if self.impulse_up:
                self.wave_history = pd.concat([self.wave_history, row_wave5, row_waveC])
            else:
                self.wave_history = pd.concat([self.wave_history, row_waveC, row_wave5])
            
            return self
        
        def update_trend(self):
            self.trend[0] = self.impulse_up
            self.trend[1] = self.start_value
            self.trend[2] = self.start_index
            return
        
        def describe(self):
            print()
            print(f'Trend')
            for attr, value in vars(self).items():
                print(f'{attr}: {value}')
            print()
            return

    class Candle:
        def __init__(self, t, data):
            self.low, self.prev_low, self.high, self.prev_high, self.stock_open, self.stock_close = self.new_candle(t, data)

        @staticmethod
        def new_candle(t, data):
            t = int(t)
            low = data.iloc[t, 4]
            prev_low = data.iloc[t-1, 4]
            high = data.iloc[t, 3]
            prev_high = data.iloc[t-1, 3]
            stock_open = data.iloc[t, 2]
            stock_close = data.iloc[t, 1]
            return low, prev_low, high, prev_high, stock_open, stock_close
        
    class DataAggregator:
        @staticmethod
        def aggregate_data(agg_range, data):
            print("In Aggregate Data")
            
            # Initialize an empty DataFrame for aggregated data
            agg_data = pd.DataFrame(columns=["Date", "Close", "Open", "High", "Low", "Volume"])
            print(f"Agg range: {agg_range}")
            
            t = len(data)
            start_value = t - agg_range 
            print(f"t: {t}, Start Value: {start_value}")

            while t > 0:
                # Select the range of data to aggregate
                candles = data.loc[start_value:t]
                print("CANDLES")
                print(candles)

                # Calculate the necessary values for the new row in agg_data
                low = candles["Low"].min()
                close = candles["Close"].iloc[-1]
                week_open = candles["Open"].iloc[0]
                high = candles["High"].max()
                volume = candles["Volume"].mean()
                date = candles["Date"].iloc[0]
                
                # Create a new DataFrame row with the calculated values
                row = pd.DataFrame({
                    "Date": [date],
                    "Close": [close],
                    "Open": [week_open],
                    "High": [high],
                    "Low": [low],
                    "Volume": [volume]
                })
                print(f"Row: {row}")
                
                # Append the new row to the aggregated data
                agg_data = pd.concat([agg_data, row], ignore_index=True)
                print("AGG DATA")
                print(agg_data)

                # Update the range for the next iteration
                t -= agg_range
                start_value = t - agg_range
                print(f"New t: {t}")

            # Calculate additional columns
            agg_data['Delta Price'] = agg_data["Close"] - agg_data["Open"]
            agg_data['Velocity'] = 0
            
            # Calculate Velocity
            for i in agg_data.index:
                delta_price = agg_data.loc[i, 'Delta Price']
                if delta_price != 0:
                    agg_data.loc[i, 'Velocity'] = (agg_data.loc[i, 'High'] - agg_data.loc[i, 'Low']) * delta_price / abs(delta_price)
            
            # Calculate Momentum and Delta Momentum
            agg_data['Momentum'] = agg_data['Volume'] * agg_data['Velocity']
            agg_data['Delta Momentum'] = agg_data['Momentum'].diff().fillna(0)

            # Drop any rows with NaN values and reset the index
            agg_data.dropna(inplace=True)
            agg_data.reset_index(drop=True, inplace=True)

            print("Final AGG DATA")
            print(agg_data)
            
            return agg_data




In [3]:
class ElliottChain():
    def __init__(self):
        self.chain = pd.DataFrame(columns=['Trend', 'Current Value', 'Temp Value', 'Wave Shape', 'Wave Count', 'Start Date', 'End Date', 'Min', 'Max', 'Ratio'])

    class NewRow:
        def __init__(self, wave, data):
            self.df = pd.DataFrame(columns=['Trend', 'Current Value', 'Temp Value', 'Wave Shape', 'Wave Count', 'Start Date', 'End Date', 'Min', 'Max', 'Ratio'])
            self.wave_history = wave.wave_history
            # self.impulse_up = wave.impulse_up
            self.wave_type = wave.wave_type 
            self.wave_identity = wave.wave_identity 
            self.proportion = wave.proportion
            self.trend_direction = wave.trend_direction
            self.wave_min = wave.wave_min
            self.wave_max = wave.wave_max
            self.value = wave.value
            self.temp_value = wave.temp_value
            self.start_date, self.end_date = self.identify_history_values(data)
            # print(self.start_date, self.end_date)

        def identify_history_values(self, data):
            first_index = self.wave_history.index[0]
            last_index = self.wave_history.index[-1]
            print(data)
            start_date = data.iloc[first_index, 0]
            end_date = data.iloc[last_index, 0]
            start_date = Data.excel_date_to_datetime(int(start_date))
            # print("Start Date")
            # print(start_date)
            end_date = Data.excel_date_to_datetime(int(end_date))
            # print("End Date")
            # print(end_date)
            # wave_min = self.wave_history.iloc[:, 0].min()
            # wave_max = self.wave_history.iloc[:, 0].max()
            return start_date, end_date #, wave_min, wave_max
        
        def create_row(self):
            new_row_data = {
                'Trend': [self.trend_direction],
                'Current Value': [self.value],
                'Temp Value': [self.temp_value],
                'Wave Shape': [self.wave_type],
                'Wave Count': [self.wave_identity],
                'Start Date': [self.start_date],
                'End Date': [self.end_date],
                'Min': [self.wave_min],
                'Max': [self.wave_max],
                'Ratio': [self.proportion]
            }

            new_row = pd.DataFrame(new_row_data)
            self.df = pd.concat([self.df, new_row])

            return self
        
    @classmethod
    def local_wave_history(cls, *new_rows):
        local_wave_history = pd.concat([row.df for row in new_rows])
        return local_wave_history
    
    def update_history(self, local_wave_history):
        wave_count = local_wave_history.shape[0]
        wave_type = local_wave_history["Wave Shape"].iloc[-1]
        # print(wave_count)
        # print("Wave Type: " + str(wave_type))

        if wave_count == 1 and wave_type != "Wave 1":
            self.chain = pd.concat([self.chain, local_wave_history], ignore_index=True)
        else: 
            self.chain = self.chain.drop(self.chain.index[-2:])
            self.chain = pd.concat([self.chain, local_wave_history], ignore_index=True)

        del local_wave_history

In [4]:
class Wave(ABC):
    def __init__(self, trend, t, data, wave_identity="A", wave_type="Indeterminate"):
        self.wave_type = wave_type
        self.wave_identity = wave_identity
        self.wave_history = pd.DataFrame(columns=["Key"])
        self.trend = trend
        self.start_value = self.trend.start_value
        self.start_index = self.trend.start_index
        # print(f'Start Index: {self.start_index}')
        # print(type(self.start_index))
        self.comparison_wave_length = self.trend.wave_length

        if self.trend.impulse_up:
            self.trend_direction = "Up"
        else:
            self.trend_direction = "Down"

        # self.wave_history, self.value, self.index, self.wave_length, self.proportion = self.calculate_wave(t, data)
        # self.check_trend, self.key_name, self.value_name, self.wave_history = self.set_trend_variables(t, data)

    def set_trend_variables_odd(self, t, data):
        # print("Set Variables Odd")
        candle = Data.Candle(t, data)
        if self.trend_direction == "Up":
            self.key_name = "Max Price"
            self.value_name = candle.high 
            if candle.low >= self.trend.start_value and ((candle.low >= candle.prev_low) or (candle.stock_open >= candle.stock_close) or (candle.high >= candle.prev_high)):
                self.check_trend = True 
            else:
                self.check_trend = False
            # print(self.trend_direction, self.key_name, self.value_name, self.check_trend)
        else:
            self.key_name = "Min Price"
            self.value_name = candle.low
            if candle.high <= self.trend.start_value and ((candle.high <= candle.prev_high) or (candle.stock_open <= candle.stock_close) or (candle.low <= candle.prev_low)):
                self.check_trend = True 
            else:
                self.check_trend = False
            # print(self.trend_direction, self.key_name, self.value_name, self.check_trend)
        
        self.wave_history.columns = [self.key_name]
        # print(f"Odd Variables: {self.check_trend, self.key_name, self.value_name, self.wave_history}")

        return self.check_trend, self.key_name, self.value_name, self.wave_history 
    
    def set_trend_variables_even(self, t, data):
        candle = Data.Candle(t, data)
        if self.trend_direction == "Up":
            self.key_name = "Min Price"
            self.value_name = candle.low 
            if ((candle.low <= candle.prev_low) or (candle.stock_close >= candle.stock_open) or (candle.high <= candle.prev_high)):
                self.check_trend = True
            else:
                self.check_trend = False
        else:
            self.key_name = "Max Price"
            self.value_name = candle.high
            if ((candle.high >= candle.prev_high) or (candle.stock_open >= candle.stock_close) or (candle.low >= candle.prev_low)):
                self.check_trend = True 
            else:
                self.check_trend = False

        self.wave_history.columns = [self.key_name]

        return self.check_trend, self.key_name, self.value_name, self.wave_history
    
    def set_trend_variables(self, t, data):
        if self.wave_identity in ["1", "3", "5", "A", "C"]:
            self.check_trend, self.key_name, self.value_name, self.wave_history = self.set_trend_variables_odd(t, data)
        else:
            self.check_trend, self.key_name, self.value_name, self.wave_history = self.set_trend_variables_even(t, data)
        return self.check_trend, self.key_name, self.value_name, self.wave_history

    def assign_value_min_max(self):
        if (self.trend_direction == "Up" and self.wave_identity in ['1', '3', '5', 'A', 'C']) or (self.trend_direction == "Down" and self.wave_identity in ['2', '4', 'B']):
            self.value = self.wave_history[self.key_name].max()
            self.wave_max = self.value
            self.wave_min = self.start_value
        else:
            self.value = self.wave_history[self.key_name].min()
            self.wave_min = self.value
            self.wave_max = self.start_value
        
        return self.value, self.wave_max, self.wave_min
    
    def terminate_end_of_dataset(self, t):
        # Instead of self.value being calculated as usual, self.value will return the last value of wave_history, to maintain integrity with index, value pairs.
        # print("Terminate end of dataset")
        self.value, self.wave_max, self.wave_min = self.assign_value_min_max()
        # print("Key: " + str(self.key_name))
        self.value = self.wave_history[self.key_name].iloc[-1]
        # print("End of Dataset Value: " + str(self.value))
        self.index = self.wave_history.index[-1]
        # print("End of Dataset Index: " + str(self.index))

        if self.key_name == "Min Price":
            self.temp_value = self.wave_min
            # print("Self.temp_value: " + str(self.temp_value))
        else:
            self.temp_value = self.wave_max
            # print("Self.temp_value: " + str(self.temp_value))
            
        self.wave_length = self.temp_value - self.start_value
        self.proportion = self.wave_length / self.comparison_wave_length
        
        self.wave_history.index.name = 'Time'
        self.trend.start_index = t
        
        # print("EOD Wave Length: " + str(self.wave_length))
        # print("EOD Proportion: " + str(self.proportion))
        # print("New trend start index: " + str(t))

        return self

    def describe(self):
        print()
        print(f'Wave {self.wave_identity}')
        for attr, value in vars(self).items():
            print(f'{attr}: {value}')
        print()
        return
    
    def update_history(self, t, data):
        new_row = pd.DataFrame({self.key_name: [self.value_name]}, index=[t])
        self.wave_history = pd.concat([self.wave_history, new_row], ignore_index=False)

        t += 1
        try:
            self.check_trend, self.key_name, self.value_name, self.wave_history = self.set_trend_variables(t, data)
        except:
            self.terminate_end_of_dataset(t)
        
        return self, t
    
    def calculate_wave(self, t, data):
        # print("--------------------")
        # print("Calculate Wave Start")
        # print(f'Wave {self.wave_identity} Start')
        # print(f"Start Index: {self.start_index}")
        # print(type(self.start_index))
        # if t == self.start_indext = int(self.start_index)
        self.check_trend, self.key_name, self.value_name, self.wave_history = self.set_trend_variables(t, data)
        # print(f'Check Trend: {self.check_trend}; Key Name: {self.key_name}; Value Name: {self.value_name}')
        try:
            self, t = self.update_history(t, data)
        except: 
            # print("Terminate pre-loop")
            self.terminate_end_of_dataset(t)
            return self.wave_history, self.value, self.index, self.wave_length, self.proportion, self.temp_value, t
        # print(f"Wave History {self.wave_identity}")
        # print(self.wave_history)

        while t < len(data):
            # print(f"Data length: {len(data)}")
            # print("Loop t: " + str(t))
            # print(t)
            # print(self.check_trend)
            if self.check_trend is True:
                try:
                    self, t = self.update_history(t, data)
                except: 
                    # print("Terminate end of dataset")
                    self.terminate_end_of_dataset(t)
                    break
                # print(f't after update history: {t}')
                # print("Loop History")
                # print(self.wave_history)
                # print(f"Trend Check: {self.check_trend}")
            else:
                # print("Update History Complete")
                self.check_trend, self.key_name, self.value_name, self.wave_history = self.set_trend_variables(t, data)
                # print(self.wave_history)

                if self.wave_history[self.key_name].isnull().any():
                    print("There are missing values in the key column.")

                self.value, self.wave_max, self.wave_min = self.assign_value_min_max()

                if self.key_name == "Min Price":
                    self.temp_value = self.wave_min
                else:
                    self.temp_value = self.wave_max
                
                self.index = self.wave_history.index[self.wave_history[self.key_name] == self.value].tolist()

                if len(self.index) > 0:
                    self.index = self.index[0]
                else:
                    print(f"No index found for Wave {self.wave_identity}.")
                
                loop = 0
                while self.index == self.start_index:
                    # print("self.index == self.start_index")
                    loop += 1
                    try:
                        self.wave_history, self.value, self.index, self.wave_length, self.proportion, self.temp_value, t = self.calculate_wave(t, data)
                        t += loop 
                        # print(f"Wave {self.wave_identity} History")
                        # print(self.wave_history)
                    except:
                        self.terminate_end_of_dataset(self.index)
                        break

                self.wave_length = self.value - self.start_value

                self.proportion = self.wave_length / self.comparison_wave_length
                
                self.wave_history.index.name = 'Time'

                mask = self.wave_history.index <= self.index
                self.wave_history = self.wave_history[mask]

                t = self.index
                break

        return self.wave_history, self.value, self.index, self.wave_length, self.proportion, self.temp_value, t

    @abstractmethod 
    def check_shape(self):
        pass 

    @abstractmethod 
    def handle_continuation(self, data):
        pass
    
    @abstractmethod
    def calculate_theoretical_impulse(self):
        pass

    def update_trend(self):
        self.wave_length = self.wave_max - self.wave_min
        if self.new_waveC:
            self.trend.wave_length = self.value - self.trend.start_value
            if self.trend_direction == "Up":
                self.trend.impulse_up = False
                self.trend.start_value = self.value 
                self.trend.start_index = self.index
            else:
                self.trend.impulse_up = True
                self.trend.start_value = self.value 
                self.trend.start_index = self.index 

            self.trend.trend = [self.trend.impulse_up, self.trend.start_value, self.trend.start_index]
            
        else:
            print("New Wave C is False")
        
        return self.trend




In [5]:
class WaveA(Wave):
    def __init__(self, trend, t, data, wave_identity="A", wave_type="Indeterminate"):
        super().__init__(trend, t, data, wave_identity, wave_type)
        # self.wave_type = wave_type
        # self.wave_identity = wave_identity
        # self.wave_history = pd.DataFrame(columns=["Key"])
        # self.trend = trend
        # self.start_value = self.trend.start_value
        # self.start_index = self.trend.start_index
        # print(f'Start Index: {self.start_index}')
        # print(type(self.start_index))
        # self.comparison_wave_length = self.trend.wave_length

        # if self.trend.impulse_up:
        #     self.trend_direction = "Up"
        # else:
        #     self.trend_direction = "Down"

        self.wave_history, self.value, self.index, self.wave_length, self.proportion, self.temp_value, t = self.calculate_wave(t, data)
        # self.check_trend, self.key_name, self.value_name, self.wave_history = self.set_trend_variables(t, data)
        
        self.ideal_impulse_chain = self.calculate_theoretical_impulse()
                    
    def calculate_theoretical_impulse(self):
        ideal_impulse_chain = pd.DataFrame(columns=['Wave Start', 'Wave 1', 'Wave 2', 'Wave 3', 'Wave 4', 'Wave 5'])
        
        small_wave2 = self.value - (self.wave_length) * 0.5
        small_wave3 = (self.wave_length) * 1.618 + small_wave2
        len_wave3_small = small_wave3 - small_wave2 
        small_wave4 = small_wave3 - len_wave3_small * 0.146
        small_wave5_options = [small_wave4 + (small_wave3 - small_wave4) * 1.236, small_wave4 + self.wave_length, small_wave4 + (small_wave3 - self.start_value) * 0.618]
        small_wave5 = min(small_wave5_options)
        small_row = pd.DataFrame({'Wave Start': [self.start_value], 'Wave 1': [self.value], 'Wave 2': [small_wave2], 'Wave 3': [small_wave3], 'Wave 4': [small_wave4], 'Wave 5': [small_wave5]})

        ideal_impulse_chain = pd.concat([ideal_impulse_chain, small_row])
        
        return ideal_impulse_chain
    
    def check_shape(self):
        return 
    
    def handle_continuation(self, data):
        data = data 
        return

In [6]:
class WaveB(Wave):
    def __init__(self, trend, t, data, waveA, wave_identity="B", wave_type="Indeterminate"):
        super().__init__(trend, t, data, wave_identity, wave_type)
        self.waveA = waveA
        self.wave_type = wave_type
        self.wave_identity = wave_identity
        self.wave_history = pd.DataFrame(columns=["Key"])
        self.trend = self.waveA.trend
        self.start_value = self.waveA.value
        self.data = data
        if self.trend.start_index == self.waveA.index:
            self.start_index = self.waveA.index + 1
        else:
            self.start_index = self.waveA.index
        self.comparison_wave_length = self.waveA.wave_length

        if self.trend.impulse_up:
            self.trend_direction = "Up"
        else:
            self.trend_direction = "Down"

        if len(data) - 1 in [self.waveA.index]:
            print("Init Wave B; Dataset aready ended.")

        # print("Describe Wave B Init")
        # self.describe()

        self.wave_history, self.value, self.index, self.wave_length, self.proportion, self.temp_value, t = self.calculate_wave(t, data)
        # self.check_trend, self.key_name, self.value_name, self.wave_history = self.set_trend_variables(t, data)

        self.wave_type = self.check_shape()
        self.waveA.ideal_impulse_chain = self.calculate_theoretical_impulse()
    
    def check_shape(self):
        if self.wave_length/abs(self.wave_length) == self.waveA.wave_length/abs(self.waveA.wave_length):
            print("This may be a continuation of Wave 1.")
            self.wave_type = "Wave 1"    
        elif (abs(self.proportion) > 0.236 and abs(self.proportion) < 0.9) or (self.index == len(self.data) - 1): 
            self.wave_type = "Indeterminate"
        elif abs(self.proportion) >= 0.9 and abs(self.proportion) <= 1.0:
            self.wave_type = "Regular Flat"
            # calculate_theoretical_flat()
        elif abs(self.proportion) > 1.0 and abs(self.proportion) <= 1.236:
            self.wave_type = "Other Flat"
        elif abs(self.proportion) > 1.236:
            print("Please review manually.")
            self.wave_type = "Manual Review"
        else:
            print("This may be a continuation of Wave 1.")
            self.wave_type = "Wave 1"

        return self.wave_type

    def calculate_theoretical_impulse(self):
        ideal_impulse_chain = self.waveA.ideal_impulse_chain
        
        return ideal_impulse_chain
    
    def handle_continuation(self, waveA, data):
        print("Handling Continuation")
        print(self.wave_type)
        initial_t = self.index
        loop = 0
        while self.wave_type == "Wave 1" and self.index < len(data) - 1:
            print("Looping")
            print(f"Wave Type: {self.wave_type}")
            conversion_entries = pd.DataFrame(columns=[self.key_name])
            conversion_index = int(self.index)
            t = self.waveA.start_index

            while t <= conversion_index:
                print("Updating to t == conversion_index")
                print(f"conversion_index: {conversion_index}")
                print(f"t: {t}")
                try:
                    self.check_trend, self.key_name, self.value_name, conversion_entries = self.set_trend_variables(t, data)
                    conversion_entries.loc[t] = {self.key_name: self.value_name}
                
                    t += 1
                except: 
                    self.terminate_end_of_dataset(t)
                    print()
                    print("Terminated Line 71")
                    break
            
            conversion_entries = conversion_entries.sort_index()
            self.waveA.wave_history = conversion_entries
            print("Conversion Entries to Wave A History 97")
            print(self.waveA.wave_history)

            t = self.index + 1

            while self.waveA.index == waveA.index and t < len(data):
                loop += 1
                try:

                    self.waveA.wave_history, self.waveA.value, self.waveA.index, self.waveA.wave_length, self.waveA.proportion, self.waveA.temp_value, t = self.waveA.calculate_wave(t, data)
                    t = initial_t + loop
                    # self.index += 1
                    print("Wave A History")
                    print(self.waveA.wave_history)
                except:
                    self.terminate_end_of_dataset(self.index)
                    break
                    
            # if t < len(data):
            #     self.waveA.wave_history, self.waveA.value, self.waveA.index, self.waveA.wave_length, self.waveA.proportion, t = self.waveA.calculate_wave(t, data)
            #     print("Wave A History")
            #     print(self.waveA.wave_history)

            # while self.waveA.index == waveA.index and self.index < len(data) - 1:
            #     print("Need to continue; Wave A is the same.")
            #     print(self.index)
            #     try:
            #         self.index += 1
            #         self.handle_continuation(self.waveA, data)
            #         # if self.index == len(data) - 1:
            #         #     self.terminate_end_of_dataset(self.index)
            #         #     print()
            #         #     print("Terminated Line 84")
            #         #     exit()
            #     except:
            #         self.terminate_end_of_dataset(self.index)
            #         print()
            #         print("Terminated Line 90")
            #         # break
            
            # self.waveA.ideal_impulse_chain = self.calculate_theoretical_impulse()
            print("Continuation Ideal Impulse Chain")
            print(self.waveA.ideal_impulse_chain)
            t = self.waveA.index
            print(f"t being passed to Wave B: {t}")

            if t < len(data) - 1:
                self = WaveB(self.trend, t, data, self.waveA, wave_identity="B", wave_type="Indeterminate")
                print("Wave B Calculated")
                self.check_shape()
                print(f"Check shape Wave B: {self.wave_type}")
            else:
                print("We elsed, kids")
                # self.waveA.terminate_end_of_dataset(t)
                self.wave_type = "Delete"  
                break
        
        return self




In [7]:
class WaveC(Wave):
    def __init__(self, trend, t, data, waveA, waveB, wave_identity="C", wave_type="Indeterminate"):
        super().__init__(trend, t, data, wave_identity, wave_type)
        self.waveA = waveA
        self.waveB = waveB
        self.wave_type = self.waveB.wave_type
        self.start_value = self.waveB.value
        if self.waveA.index == self.waveB.index:
            self.start_index = self.waveB.index + 1
        else:
            self.start_index = self.waveB.index
        self.comparison_wave_length = self.waveA.wave_length
        self.trend_direction = self.waveB.trend_direction
        self.data_length = len(data)

        if self.data_length - 1 in [self.waveA.index, self.waveB.index]:
            print("Init Wave C; Dataset aready ended.")

        self.check_trend, self.key_name, self.value_name, self.wave_history = self.set_trend_variables(t, data)
        self.wave_history, self.value, self.index, self.wave_length, self.proportion, self.temp_value, t = self.calculate_wave(t, data)

        self.wave_type = self.check_shape()
        self.waveA.ideal_impulse_chain = self.calculate_theoretical_impulse()
    
    def check_shape(self):
        if self.wave_type == "Regular Flat" and abs(self.proportion) <= 1.236:
            self.wave_type
            self.new_waveC = True
        elif abs(self.waveB.proportion) > 0.9 and abs(self.waveB.proportion) <= 1.236 and abs(self.proportion) >= 0.618 and abs(self.proportion) < 1.236:
            print("This is a Running Flat Wave C.")
            self.wave_type = "Running Flat" 
            self.new_waveC = True
        elif abs(self.waveB.proportion) >= 1.236 and abs(self.waveB.proportion) <= 1.618 and abs(self.proportion) >= 1.236 and abs(self.proportion) <= 1.618:
            print("This is an Expanded Flat Wave C.")
            self.wave_type = "Expanded Flat"
            self.new_waveC = True
        elif abs(self.proportion) >= 0.618 and abs(self.proportion) < 1.236:
            print("This is a Zig Zag Wave C.")
            self.new_waveC = True
            # the waves should be class instances so they can have properties
            self.wave_type = "Zig Zag"
            # Future: check ElliottWaveChain for triangles
        elif abs(self.proportion) >= 1.236 and abs(self.proportion) < 1.618:
            print("This is a Diagonal Wave 3.")
            self.new_waveC = False
            self.wave_type = "Diagonal"
            self.waveA.wave_identity = "1" 
            self.waveB.wave_identity = "2" 
            self.wave_identity = "3"
            print(self.waveA.wave_identity, self.waveB.wave_identity, self.wave_identity)
            # differentiate between impule and diagonal
            # recalculate theoretical impulse waves 4 + 5
        elif abs(self.proportion) >= 1.618:
            print("This is an Impulse Wave 3.")
            self.wave_type = "Impulse"
            self.waveA.wave_identity = "1" 
            self.waveB.wave_identity = "2" 
            self.wave_identity = "3"
            print(self.waveA.wave_identity, self.waveB.wave_identity, self.wave_identity)
            self.new_waveC = False
        elif abs(self.proportion) < 0.618 and self.index == (self.data_length - 1):
            self.wave_type = "Wave C or 3"
            self.new_waveC = False
        elif abs(self.proportion) < 0.618 and self.value >= self.waveA.value:
            self.wave_type = "Wave 1" 
            self.new_waveC = False
        else:  
            self.wave_type = "Wave B Continuation"
            self.new_waveC = False

        self.waveA.wave_type = self.wave_type 
        self.waveB.wave_type = self.wave_type
        
        return self.wave_type, self.waveA, self.waveB, self.wave_identity, self.new_waveC

    def calculate_theoretical_impulse(self):
        ideal_impulse_chain = pd.DataFrame(columns=['Wave Start', 'Wave 1', 'Wave 2', 'Wave 3', 'Wave 4', 'Wave 5'])
        
        small_wave2 = self.waveB.value
        small_wave3 = (self.waveA.wave_length) * 1.618 + small_wave2
        len_wave3_small = small_wave3 - small_wave2 
        small_wave4 = small_wave3 - len_wave3_small * 0.146
        small_wave5_options = [small_wave4 + (small_wave4 - small_wave2) * 1.236, small_wave4 + self.waveA.wave_length, small_wave4 + (small_wave3 - self.waveA.start_value) * 0.618]
        small_wave5 = min(small_wave5_options)
        small_row = pd.DataFrame({'Wave Start': [self.waveA.start_value], 'Wave 1': [self.value], 'Wave 2': [small_wave2], 'Wave 3': [small_wave3], 'Wave 4': [small_wave4], 'Wave 5': [small_wave5]})

        ideal_impulse_chain = pd.concat([ideal_impulse_chain, small_row])
        
        return ideal_impulse_chain
    
    def handle_continuation(self, waveB, data):
        print("Handling Continuation")
        while self.index < len(data) and self.wave_type == "Wave 1":
            print("Wave 1 Continuation from Wave C")
            self.waveB = self.waveB.handle_continuation(self.waveA, data)
            
            if self.waveB.wave_type == "Delete":
                self.waveB = None
                self.wave_type = "Delete"
                break

            else:
                if t < len(data) - 1:
                    self = WaveC(self.waveB.trend, t, self.waveA, self.waveB, data)
                    self.check_shape()
                    print(f"Updated Wave C Shape: {self.check_shape()}")
                else:
                    self.wave_type = "Delete"
                    break


        
        # if self.waveB is not None:
        #     print("Wave B is Not None")
        #     t = self.waveB.index 
        #     print(f"t passed to Wave C: {t}")
        #     if t < len(data) - 1:
        #         self = WaveC(self.waveB.trend, t, data, self.waveA, self.waveB, data)
        #         self.check_shape()
        #     elif self.wave_type == "Wave 1" or self.wave_type == "Wave B Continuation":
        #         self.index += 1
        #         self.handle_continuation(self.waveB, data)
        #     # elif self.start_index == self.index:
        #     #     t += 1 
        #     #     self.handle_continuation(self.waveB, data)
        #     else:
        #         self.wave_type = "Delete"
        #         print("Delete Wave C")

        #     print("Wave C Continuation Type")
        #     print(self.wave_type)
        # else:
        #     self.wave_type = "Delete"
            
        initial_t = self.index
        loop = 0

        while self.wave_type != "Delete" and self.index < len(data) - 1 and self.wave_type in ["Wave 1", "Wave B Continuation"]:
            print("Looping")
            print(f"Wave Type: {self.wave_type}")
            conversion_entries = pd.DataFrame(columns=[self.key_name])
            conversion_index = int(self.index)
            t = self.waveB.start_index

            while t <= conversion_index:
                print("Updating to t == conversion_index")
                print(f"conversion_index: {conversion_index}")
                print(f"t: {t}")
                try:
                    self.check_trend, self.key_name, self.value_name, conversion_entries = self.set_trend_variables(t, data)
                    conversion_entries.loc[t] = {self.key_name: self.value_name}
                
                    t += 1
                except: 
                    self.terminate_end_of_dataset(t)
                    print()
                    print("Terminated Line 71")
                    break
            
            conversion_entries = conversion_entries.sort_index()
            self.waveB.wave_history = conversion_entries
            print("Conversion Entries to Wave A History 161")
            print(self.waveA.wave_history)

            t = self.index + 1

            while self.waveB.index == waveB.index and t < len(data):
                loop += 1
                try:

                    self.waveB.wave_history, self.waveB.value, self.waveB.index, self.waveB.wave_length, self.waveB.proportion, self.waveB.temp_value, t = self.waveB.calculate_wave(t, data)
                    t = initial_t + loop
                    # self.index += 1
                    print("Wave A History")
                    print(self.waveA.wave_history)
                except:
                    self.terminate_end_of_dataset(self.index)
                    break
            
            # self.waveA.ideal_impulse_chain = self.calculate_theoretical_impulse()
            print("Continuation Ideal Impulse Chain")
            print(self.waveA.ideal_impulse_chain)
            t = self.waveB.index
            print(f"t being passed to Wave C: {t}")

            if t < len(data) - 1:
                self = WaveC(self.trend, t, data, self.waveA, wave_identity="B", wave_type="Indeterminate")
                print("Wave B Calculated")
                self.check_shape()
                print(f"Check shape Wave B: {self.wave_type}")
            else:
                print("We elsed, kids")
                # self.waveA.terminate_end_of_dataset(t)
                self.wave_type = "Delete"  
                break
        

        return self  

        

In [8]:
class Wave4(Wave):
    def __init__(self, trend, t, data, waveA, waveB, waveC, wave_identity="4", wave_type="Indeterminate"):
        super().__init__(trend, t, data, wave_identity, wave_type)
        self.waveA = waveA
        self.waveB = waveB
        self.waveC = waveC
        self.wave_type = self.waveC.wave_type
        self.start_value = self.waveC.value
        if self.waveB.index == self.waveC.index:
            self.start_index = self.waveC.index + 1
        else:
            self.start_index = self.waveC.index
        self.comparison_wave_length = self.waveA.wave_length
        self.trend_direction = self.waveC.trend_direction

        if len(data) - 1 in [self.waveA.index, self.waveB.index, self.waveC.index]:
            print("Init Wave 4; Dataset aready ended.")

        self.check_trend, self.key_name, self.value_name, self.wave_history = self.set_trend_variables(t, data)
        self.wave_history, self.value, self.index, self.wave_length, self.proportion, self.temp_value, t = self.calculate_wave(t, data)

        self.wave_type = self.check_shape()
        self.waveA.ideal_impulse_chain = self.calculate_theoretical_impulse()
    
    def check_shape(self):
        if self.value <= self.waveA.value or abs(self.proportion) > 0.5:
            print("This is a Diagonal Wave 4.")
            self.wave_type = "Diagonal"
        elif abs(self.proportion) <= 0.146:
            print("This may be a continuation of Wave 3.")  
            self.wave_type = "Wave 3"
        elif abs(self.proportion) > 1:
            self.wave_type = "Indeterminate"
        else: 
            print("This is an Impulse Wave 4.")
            self.wave_type = "Impulse"

        self.waveA.wave_type = self.wave_type 
        self.waveB.wave_type = self.wave_type
        self.waveC.wave_type = self.wave_type
        
        return self.wave_type

    def calculate_theoretical_impulse(self):
        ideal_impulse_chain = pd.DataFrame(columns=['Wave Start', 'Wave 1', 'Wave 2', 'Wave 3', 'Wave 4', 'Wave 5'])
        
        small_wave2 = self.waveB.value
        small_wave3 = self.waveC.value
        len_wave3_small = small_wave3 - small_wave2 
        small_wave4 = small_wave3 - len_wave3_small * 0.146
        small_wave5_options = [small_wave4 + (small_wave3 - small_wave4) * 1.236, small_wave4 + self.waveA.wave_length, small_wave4 + (small_wave3 - self.waveA.start_value) * 0.618]
        small_wave5 = min(small_wave5_options)
        small_row = pd.DataFrame({'Wave Start': [self.waveA.start_value], 'Wave 1': [self.waveA.value], 'Wave 2': [small_wave2], 'Wave 3': [small_wave3], 'Wave 4': [small_wave4], 'Wave 5': [small_wave5]})

        ideal_impulse_chain = pd.concat([ideal_impulse_chain, small_row])
        
        return ideal_impulse_chain

    def handle_continuation(self, data):
        data = data 
        return

In [9]:
class Wave5(Wave):    
    def __init__(self, trend, t, data, waveA, waveB, waveC, wave4, wave_identity="5", wave_type="Indeterminate"):
        super().__init__(trend, t, data, wave_identity, wave_type)
        self.waveA = waveA
        self.waveB = waveB
        self.waveC = waveC
        self.wave4 = wave4
        self.wave_type = self.wave4.wave_type
        self.start_value = self.wave4.value
        if self.waveC.index == self.wave4.index:
            self.start_index = self.wave4.index + 1
        else:
            self.start_index = self.wave4.index
        self.comparison_wave_length = self.waveA.wave_length
        self.trend_direction = self.wave4.trend_direction
        self.data = data
        self.momentum_std_dev = self.data['Delta Momentum'].std()

        if len(data) - 1 in [self.waveA.index, self.waveB.index, self.waveC.index, self.wave4.index]:
            print("Init Wave 5; Dataset aready ended.")

        # self.check_trend, self.key_name, self.value_name, self.wave_history = self.set_trend_variables(t, data)
        self.wave_history, self.value, self.index, self.wave_length, self.proportion, self.temp_value, t = self.calculate_wave(t, data)

        self.wave_type = self.check_shape()
    
    def calculate_end_conditions(self):
        condition_wave4_min = self.wave4.wave_length * 1.236 + self.wave4.value 
        condition_wave4_max = self.wave4.wave_length * 1.618 + self.wave4.value 
        condition_wave1 = self.waveA.wave_length + self.wave4.value
        condition_wave3 = (self.waveC.value - self.waveA.start_value) * 0.618 + self.wave4.value 

        conditions = {'Wave 4 Min' : condition_wave4_min,
                      'Wave 4 Max' : condition_wave4_max,
                      'Wave 5 = Wave 1' : condition_wave1,
                      'Retrace Waves 1-3': condition_wave3}

        conditions = pd.DataFrame(conditions.items(), columns=['Condition', 'Value'])
        self.conditions = conditions.sort_values('Value').reset_index(drop=True)

        # print("Conditions")
        # print(self.conditions)
        return self
    
    def check_shape(self):
        self.calculate_end_conditions()

        # momentum_table = self.data[self.start_index : self.index + 1]
        # self.momentum_std_dev = momentum_table['Delta Momentum'].std()
        

        print(f"Length Conditions: {len(self.conditions)}")
        print(self.conditions.index)

        i = 0
        while i < len(self.conditions):
            condition_value = self.conditions['Value'].iloc[i]
            condition_type = self.conditions['Condition'].iloc[i]
            
            if (self.key_name == "Min Price" and self.value < condition_value) or \
            (self.key_name != "Min Price" and self.value > condition_value):
                self.wave_type = condition_type
                break  # Exit the loop once the condition is met
            
            i += 1
        else:
            # If the loop completes without breaking (no condition met), set wave_type to "Extension"
            self.wave_type = "Extension"
        
        if self.momentum_std_dev != 0:  # Check if momentum_std_dev is not zero to avoid division by zero
            momentum_check = self.data['Delta Momentum'].iloc[self.index] / self.momentum_std_dev
        else:
            momentum_check = 0
        
        print(f"Momentum Check: {momentum_check}")

        if abs(momentum_check) < 1 and self.wave_length < self.waveC.wave_length:
            self.new_waveC = False
        else:
            self.new_waveC = True
        
        return self

    def calculate_theoretical_impulse(self):
        ideal_impulse_chain = pd.DataFrame(columns=['Wave Start', 'Wave 1', 'Wave 2', 'Wave 3', 'Wave 4', 'Wave 5'])
        
        small_wave2 = self.waveB.value
        small_wave3 = self.waveC.value
        len_wave3_small = small_wave3 - small_wave2 
        small_wave4 = self.wave4.value
        small_wave5_options = [small_wave4 + (small_wave3 - small_wave4) * 1.236, small_wave4 + self.waveA.wave_length, small_wave4 + (small_wave3 - self.waveA.start_value) * 0.618]
        small_wave5 = self.conditions['Value'].iloc[-1]
        small_row = pd.DataFrame({'Wave Start': [self.waveA.start_value], 'Wave 1': [self.waveA.value], 'Wave 2': [small_wave2], 'Wave 3': [small_wave3], 'Wave 4': [small_wave4], 'Wave 5': [small_wave5]})

        ideal_impulse_chain = pd.concat([ideal_impulse_chain, small_row])
        
        return ideal_impulse_chain

    def handle_continuation(self, data):
        print("Handling Continuation")
        print(self.wave_type)
        # conversion_index = int(self.index)
        initial_t = self.index
        loop = 0
        t = initial_t

        while ((self.index == self.start_index) or (self.new_waveC is False)) and t < len(data) - 1:
            loop += 1
            print(f"loop: {loop}")
            try:
                t = initial_t + loop
                self.wave_history, self.value, self.index, self.wave_length, self.proportion, self.temp_value, t = self.calculate_wave(t, data)
                print(f"Wave 5 Continuation t: {t}")
                print(f"Wave {self.wave_identity} History")
                print(self.wave_history)
            except:
                self.terminate_end_of_dataset(t)
                break

            self.check_shape()
            # print(f"New Wave C: {self.new_waveC}")
            self.update_trend()

        # while self.new_waveC is False and t < len(data):
        #     print(f"Wave Type: {self.wave_type}")
        #     if t < len(data):
        #         self.wave_history, self.value, self.index, self.wave_length, self.proportion, t = self.calculate_wave(t, data)
        #         print("Wave 5 History")
        #         print(self.wave_history)
            
        #     self.check_shape()
        #     print(self.wave_type)
        #     t += 1
        #     # loop += 1
        #     print(self.new_waveC, self.value, self.index)
        
        # if t == len(data) - 1:
        #     self.terminate_end_of_dataset(t)

            
        return self


In [10]:
def try_impulse_up(data, trend, elliott_wave_history):
    t = int(trend.start_index)
    waveA = None
    waveB = None
    waveC = None
    wave4 = None
    wave5 = None
    
    # Wave A
    if t < len(data):
        waveA = WaveA(trend, t, data)
        trend_row = ElliottChain.NewRow(waveA, data)
        trend_row.create_row()
        local_wave_history = ElliottChain.local_wave_history(trend_row)
        elliott_wave_history.update_history(local_wave_history)
        # waveA.describe()
        print("Elliott Wave History")
        print(elliott_wave_history.chain)

    # Wave B
    if waveA.index < len(data) - 1:
        t = waveA.index
        print("--------------------")
        print("Wave A Complete; Start Wave B")
        waveB = WaveB(trend, t, data, waveA)
        print("Wave B Initialization Complete")
        # waveB.describe()
        waveB.check_shape()
        print(f"Wave B Shape: {waveB.wave_type}")
        print("Start Wave A Continuation")
        waveB.handle_continuation(waveA, data)
        waveA = waveB.waveA
        # print("--------------------")
        # waveA.describe()
        # print("--------------------")
        # waveB.describe()
        # print("--------------------")
        # print("Elliott Wave History")
        # elliott_wave_history.chain
        # print("--------------------")

        
        if waveB.wave_type == "Delete":
            waveB = None
        else:
            trend_row = ElliottChain.NewRow(waveB, data)
            trend_row.create_row()
            local_wave_history = ElliottChain.local_wave_history(trend_row)
            elliott_wave_history.update_history(local_wave_history)

    # Wave C
    if waveB is not None and waveB.index < len(data) - 1:
        # print("Start Wave C")
        t = waveB.index
        # print(f"t: {t}")
        waveC = WaveC(trend, t, data, waveA, waveB)
        waveC.check_shape()
        # print(f"Wave C Check Shape: {waveC.wave_type}")
        waveC = waveC.handle_continuation(waveB, data)

        waveA = waveC.waveA
        waveB = waveC.waveB 

        if waveC.wave_type == "Delete":
            waveC = None
            print("Wave C deleted")
        else:
            waveC.check_shape()

            if waveA.trend != trend:
                trend = waveA.trend
            # print("Trend Updated")
            # print("--------------------")
            # waveA.describe()
            # print("--------------------")
            # waveB.describe()
            # print("--------------------")
            # waveC.describe()
            # print("--------------------")
            # print("Elliott Wave History")
            # elliott_wave_history.chain
            # print("--------------------")


        waveA_trend_row = ElliottChain.NewRow(waveA, data)
        waveA_trend_row.create_row()

        if waveB is not None:
            waveB_trend_row = ElliottChain.NewRow(waveB, data)
            waveB_trend_row.create_row()

            if waveC is not None:
                trend = waveC.update_trend()
                # print("Updated Trend: " + str(trend.trend))
                waveC_trend_row = ElliottChain.NewRow(waveC, data)
                waveC_trend_row.create_row()
                local_wave_history = ElliottChain.local_wave_history(waveA_trend_row, waveB_trend_row, waveC_trend_row)
            else:
                local_wave_history = ElliottChain.local_wave_history(waveA_trend_row, waveB_trend_row)
        else:
            local_wave_history = ElliottChain.local_wave_history(waveA_trend_row)
        
        # print("Local Wave Histroy")
        # print(local_wave_history)

        elliott_wave_history.update_history(local_wave_history)
        # print(elliott_wave_history.chain)

    # Wave 4
    if waveC is not None and waveC.index < len(data) - 1 and waveC.wave_type in ['Diagonal', 'Impulse']:
        t = waveC.index 
        wave4 = Wave4(trend, t, data, waveA, waveB, waveC)
        # wave4.describe()
        # print("--------------------")
        # print("Elliott Wave History")
        # elliott_wave_history.chain
        # print("--------------------")
        wave4.check_shape()
        wave4_trend_row = ElliottChain.NewRow(wave4, data)
        wave4_trend_row.create_row()
        local_wave_history = ElliottChain.local_wave_history(wave4_trend_row)
        elliott_wave_history.update_history(local_wave_history)
        # print()
        # print(elliott_wave_history.chain)
        # print()
    else:
        wave4 = None

    # Wave 5
    if wave4 is not None and wave4.index < len(data) - 1:
        print("Start Wave C")
        t = wave4.index
        # print(f"t: {t}")
        wave5 = Wave5(trend, t, data, waveA, waveB, waveC, wave4)
        wave5.check_shape()
        print(f"Wave 5 Check Shape: {wave5.wave_type}")
        wave5 = wave5.handle_continuation(data)
        # print("Impulse Wave Up Line 125 Complete")

        waveA = wave5.waveA
        waveB = wave5.waveB 
        waveC = wave5.waveC
        wave4 = wave5.wave4

        if wave5.wave_type == "Delete":
            wave5 = None
            print("Wave 5 deleted")
        else:
            # wave5.check_shape()

            # if wave5.trend != trend:
            #     trend = wave5.trend

            # wave5.handle_continuation(data)

            # wave5.check_shape()

            if wave5.trend != trend:
                trend = wave5.trend

            # print("Trend")
            # trend.describe()

            # print("Wave 5 Trend")
            # wave5.trend.describe()
            
            trend = wave5.update_trend()
            # trend.update_trend()
            # print("Updated Trend")
            # trend.describe()
            # print()
            # print(f"Trend pre-update: {trend.start_index}, {trend.start_value}, {trend.trend_direction}")

            # # if wave5.trend != trend:
            # #     trend = wave5.trend
            # #     wave4.trend = trend 
            # #     waveC.trend = trend 
            # #     waveB.trend = trend 
            # #     waveA.trend = trend
            
            # print(f"Trend post-update: {trend.start_index}, {trend.start_value}, {trend.trend_direction}")
            # print()

            # print("Trend Updated")
            # print("--------------------")
            # waveA.describe()
            # print("--------------------")
            # waveB.describe()
            # print("--------------------")
            # waveC.describe()
            # print("--------------------")
            # wave4.describe()
            # print("--------------------")
            # wave5.describe()
            # print("--------------------")
            # print("Elliott Wave History")
            # elliott_wave_history.chain
            # print("--------------------")

            trend = wave5.update_trend()
            wave5_trend_row = ElliottChain.NewRow(wave5, data)
            wave5_trend_row.create_row()
            local_wave_history = ElliottChain.local_wave_history(wave5_trend_row)
            elliott_wave_history.update_history(local_wave_history)

    return trend, elliott_wave_history, waveA.ideal_impulse_chain
            

In [11]:
class StockHistory:
    def __init__(self):
        self.stock_history = pd.DataFrame(columns=["Symbol", "Day - Current Wave Shape", "Day - Current Wave Count", "Day - Temp Count", "Day - Trend", "Day - Current Value", "Day - Temp Value", \
                                                   "Week - Current Wave Shape", "Week - Current Wave Count", "Week - Temp Count", "Week - Trend", "Week - Current Value", "Week - Temp Value", \
                                                   "Month - Current Wave Shape", "Month - Current Wave Count", "Month - Temp Count", "Month - Trend", "Month - Current Value", "Month - Temp Value", \
                                                   "Wave Start", "Wave 1", "Wave 2", "Wave 3", "Wave 4", "Wave 5", \
                                                   "Week Wave Start", "Week Wave 1", "Week Wave 2", "Week Wave 3", "Week Wave 4", "Week Wave 5", \
                                                   "Month Wave Start", "Month Wave 1", "Month Wave 2", "Month Wave 3", "Month Wave 4", "Month Wave 5"])

    @staticmethod
    def sector_lookup(symbol):
        industry = pd.read_excel('../source/industry.xlsx')
        sector = industry.loc[industry['Symbol'] == symbol, 'Sector'].values[0]
        return sector
    
    def get_table_values(self, elliott_chain):
        impulse_wave_order = ['A', 'B', '3', '4', '5', 'A']
        corrective_wave_order = ['A', 'B', 'C', 'A']

        wave_shape = elliott_chain.chain["Wave Shape"].iloc[-1]
        wave_count = elliott_chain.chain["Wave Count"].iloc[-1]
        wave_direction = elliott_chain.chain["Trend"].iloc[-1]
        wave_value = elliott_chain.chain["Current Value"].iloc[-1]
        wave_temp_value = elliott_chain.chain["Temp Value"].iloc[-1]

        if wave_temp_value == wave_value:
            temp_count = wave_count
        elif wave_shape in ['Impulse', 'Diagonal']:
            if (wave_count in ['A', '3', '5'] and wave_direction == 'Up') or (wave_count in ['B', '4'] and wave_direction == 'Down') and wave_temp_value > wave_value:
                index = impulse_wave_order.index(wave_count)
                temp_count = impulse_wave_order[index + 1]
            elif (wave_count in ['A', '3', '5'] and wave_direction == 'Down') or (wave_count in ['B', '4'] and wave_direction == 'Up') and wave_temp_value < wave_value:
                index = impulse_wave_order.index(wave_count)
                temp_count = impulse_wave_order[index + 1]
        else:
            if (wave_count in ['A', 'C'] and wave_direction == 'Up') or (wave_count in ['B'] and wave_direction == 'Down') and wave_temp_value > wave_value:
                index = corrective_wave_order.index(wave_count)
                temp_count = corrective_wave_order[index + 1]
            elif (wave_count in ['A', 'C'] and wave_direction == 'Down') or (wave_count in ['B'] and wave_direction == 'Up') and wave_temp_value < wave_value:
                index = corrective_wave_order.index(wave_count)
                temp_count = corrective_wave_order[index + 1]
        
        timescale_wave_data = pd.DataFrame({"Current Wave Shape": [wave_shape],
                                            "Current Wave Count": [wave_count],
                                            "Temp Count": [temp_count], 
                                            "Trend": [wave_direction], 
                                            "Current Value": [wave_value],
                                            "Temp Value": [wave_temp_value]}, index=[0])
        
        return timescale_wave_data
        
          
    def create_row(self, symbol, elliott_chain, week_elliott_chain, month_elliott_chain, impulse_chain, week_impulse_chain, month_impulse_chain):
        day = self.get_table_values(elliott_chain)
        day.columns = ["Day - Current Wave Shape", "Day - Current Wave Count", "Day - Temp Count", "Day - Trend", "Day - Current Value", "Day - Temp Value"]
        week = self.get_table_values(week_elliott_chain)
        week.columns = ["Week - Current Wave Shape", "Week - Current Wave Count", "Week - Temp Count", "Week - Trend", "Week - Current Value", "Week - Temp Value"]
        month = self.get_table_values(month_elliott_chain)
        month.columns = ["Month - Current Wave Shape", "Month - Current Wave Count", "Month - Temp Count", "Month - Trend", "Month - Current Value", "Month - Temp Value"]

        row = pd.DataFrame({"Symbol": [symbol]}, index=[0])

        row = pd.concat([row, day, week, month, impulse_chain, week_impulse_chain, month_impulse_chain], axis=1)
        row.columns = ["Symbol", "Day - Current Wave Shape", "Day - Current Wave Count", "Day - Temp Count", "Day - Trend", "Day - Current Value", "Day - Temp Value", \
                       "Week - Current Wave Shape", "Week - Current Wave Count", "Week - Temp Count", "Week - Trend", "Week - Current Value", "Week - Temp Value", \
                       "Month - Current Wave Shape", "Month - Current Wave Count", "Month - Temp Count", "Month - Trend", "Month - Current Value", "Month - Temp Value", \
                       "Wave Start", "Wave 1", "Wave 2", "Wave 3", "Wave 4", "Wave 5", \
                       "Week Wave Start", "Week Wave 1", "Week Wave 2", "Week Wave 3", "Week Wave 4", "Week Wave 5", \
                       "Month Wave Start", "Month Wave 1", "Month Wave 2", "Month Wave 3", "Month Wave 4", "Month Wave 5"]
        self.stock_history = pd.concat([self.stock_history, row], ignore_index=True)

        return self

    # def filter_opps(self):

In [12]:
def main():
    folder_path = '../data'

    stocks, corrupt_files = Data.create_filepath(folder_path)
    stock_history = StockHistory()

    # for testing
    # stocks = stocks.iloc[:50]

    stocks.to_excel('/Users/beatricewoodrow/Desktop/elliott_waves/good_stocks.xlsx')
    corrupt_files.to_excel('/Users/beatricewoodrow/Desktop/elliott_waves/corrupt_files.xlsx')
    
    for value in stocks["Filepath"]:
        index = stocks["Filepath"].index[stocks["Filepath"] == value]
        symbol = stocks["Symbol"].iloc[index].to_string(index=False)[:-5]
        # print()
        # print("-----------------------")
        # print(symbol)
        # print()
        try: 
            data = Data.load_data(value)[0]
            trend = Data.Trend(data)
            trend.set_initial_history()
            print("THIS IS DATA")
            print(data)
            elliott_wave_history = ElliottChain()
            # print("Creating trend row.")
            trend_row = ElliottChain.NewRow(trend, data)
            trend_row.create_row()
            # print("Establishing local history")
            local_wave_history = ElliottChain.local_wave_history(trend_row)
            # print("Updating history")
            elliott_wave_history.update_history(local_wave_history)

            week_agg = 5
            week_data = Data.DataAggregator.aggregate_data(week_agg, data)
            week_trend = Data.Trend(week_data)
            week_trend.set_initial_history()
            print("WEEK DATA")
            print(week_data)
            week_elliott_wave_history = ElliottChain()
            trend_row = ElliottChain.NewRow(week_trend, week_data)
            trend_row.create_row()
            local_wave_history = ElliottChain.local_wave_history(trend_row)
            week_elliott_wave_history.update_history(local_wave_history)

            month_agg = 22
            month_data = Data.DataAggregator.aggregate_data(month_agg, data)
            month_trend = Data.Trend(month_data)
            month_trend.set_initial_history()
            print("MONTH DATA")
            print(month_data)
            month_elliott_wave_history = ElliottChain()
            trend_row = ElliottChain.NewRow(month_trend, month_data)
            trend_row.create_row()
            local_wave_history = ElliottChain.local_wave_history(trend_row)
            month_elliott_wave_history.update_history(local_wave_history)

        # print(data.head())
        # print("Data Type: " + str(type(data)))
        except:
            continue
        # print(trend.trend)

        # print()
        # print("Initiating Elliott Chain")

        # print()
        # print("Trend Start Index: " + str(t))

        try:
            # Day Level Analysis
            t = trend.start_index
            while t < len(data) - 1:
                trend, elliott_wave_history, impulse_chain = try_impulse_up(data, trend, elliott_wave_history)
                # if manual_review_flag is True:
                #     print(f'{symbol} has been skipped.')
                #     break  # Skip to the next file
                # else:
                t = int(trend.start_index)
                # print("Next t: " + str(t))
                # print(elliott_wave_history.chain)
            clear_output(wait=False)
            
            # Week Level Analysis
            t = week_trend.start_index
            while t < len(week_data) - 1:
                week_trend, week_wave_history, week_impulse_chain = try_impulse_up(week_data, week_trend, week_elliott_wave_history)
                t = int(week_trend.start_index)
            clear_output(wait=False)
            
            # Month Level Analysis
            t = month_trend.start_index
            while t < len(month_data) - 1:
                month_trend, month_wave_history, month_impulse_chain = try_impulse_up(month_data, month_trend, month_elliott_wave_history)
                t = int(month_trend.start_index)
            clear_output(wait=False)
            
            
        
            output_filepath = f'../stocks/{symbol}.csv'
            elliott_wave_history.chain.to_csv(output_filepath, index=True)
            
            # print("Main while loop has exited.")
            # print("Impulse Chain")
            # print(impulse_chain)
            
            # if manual_review_flag is False:
            stock_history.create_row(symbol, elliott_wave_history, week_wave_history, month_wave_history, impulse_chain, week_impulse_chain, month_impulse_chain)

            # print()
            # print("---------------------")     
            # print("Stock History")
            # print(stock_history.stock_history)
            # print("---------------------") 
            
            stock_history.stock_history.to_excel('/Users/beatricewoodrow/Desktop/elliott_waves/stock_history/stock_history.xlsx', index=True)
            del trend 
            del elliott_wave_history 
            del week_wave_history
            del month_wave_history
            del impulse_chain
            del week_impulse_chain
            del month_impulse_chain
            del data 
            clear_output(wait=False)

            # sleep(0.2)
        
        except:
            stock_history

    clear_output(wait=False)
    
    return stock_history

if __name__ == "__main__":
    stock_history = main()

/var/folders/lz/688vwzb10dl5n38_qdr537lr0000gn/T/ipykernel_43820/1555583298.py:39: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '-1.875' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[i, 'Velocity'] = (df.loc[i, 'High'] - df.loc[i, 'Low']) * df.loc[i, 'Delta Price'] / abs(df.loc[i, 'Delta Price'])
/var/folders/lz/688vwzb10dl5n38_qdr537lr0000gn/T/ipykernel_43820/2237406784.py:52: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.df = pd.concat([self.df, new_row])
/var/folders/lz/688vwzb10dl5n38_qdr537lr0000gn/T/ipykernel_43820/2237406784.py:68: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA en

In [ ]:
# def filter_stock_history(stock_history):
#     # Filter Conditions:
#     # # Stock volume > median market volume -- need to add volume to stock history
#     # # Trend up:
#     # # # Current Wave Count != Temp Wave Count:
#     # # # # 

In [ ]:
stock_history.stock_history

,Symbol,Day - Current Wave Shape,Day - Current Wave Count,Day - Temp Count,Day - Trend,Day - Current Value,Day - Temp Value,Week - Current Wave Shape,Week - Current Wave Count,Week - Temp Count,...,Week Wave 2,Week Wave 3,Week Wave 4,Week Wave 5,Month Wave Start,Month Wave 1,Month Wave 2,Month Wave 3,Month Wave 4,Month Wave 5
0,CELC,Indeterminate,B,C,Down,16.030,19.7250,Indeterminate,B,C,...,9.4945,13.071898,12.549598,13.195161,8.389,11.6400,9.35450,16.750378,15.670580,17.005210
1,APXI,Indeterminate,A,A,Down,11.500,11.5000,Indeterminate,B,B,...,10.4300,12.274520,12.005220,12.338075,9.860,10.7701,10.31505,11.787592,11.572601,11.838330
2,ACRE,Wave C or 3,C,A,Up,6.705,7.0500,Wave 1,A,A,...,10.5800,6.567360,7.153205,2.917687,11.240,8.9850,11.20500,7.014380,7.626211,3.202827
3,BOLD,Impulse,3,3,Up,12.260,12.2600,Indeterminate,A,A,...,11.8750,22.764140,21.174326,23.139336,8.510,15.2400,11.87500,22.764140,21.174326,23.139336
4,CDTX,Indeterminate,A,B,Down,11.650,10.5000,Wave B Continuation,B,C,...,12.5000,35.792728,32.391990,46.787990,10.004,27.6000,18.80200,47.272328,43.115660,48.253302
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1847,CRTO,Indeterminate,B,B,Down,39.100,39.1000,Wave 1,A,B,...,27.2800,39.544440,37.753832,45.333832,22.100,35.2850,28.69250,50.025830,46.911164,50.760891
1848,AEVA,Wave 1,A,B,Down,3.280,2.6420,Indeterminate,A,A,...,6.6750,3.681700,4.118722,1.697212,2.325,7.6000,4.96250,13.497450,12.251347,13.791530
1849,ERO,Wave 1,A,B,Down,22.110,19.2400,Indeterminate,A,A,...,20.3100,7.139480,9.062376,-1.592265,24.380,16.2400,20.31000,7.139480,9.062376,-1.592265
1850,FNA,Indeterminate,A,B,Up,7.930,7.9400,Indeterminate,A,A,...,18.0700,15.060520,15.499904,13.065305,7.250,19.0000,7.95000,20.149720,18.368561,25.908561
